# TREC 2019 Precision Medicine

This is an example notebook to get the grip of TREC and check everything's running.
Remember you need to place a **config.json** file under the **trec_utils** directory pointing to the elasticsearch server. 

In [3]:
import json
from json2html import *
from IPython.display import HTML
import pandas

import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from trec_utils import utils, running, evaluation

In [4]:
config = utils.load_config()

Here an example from 2017, where we load and list the topics (with already some preprocessing, like splitting the genes and calculating MeSH age groups).

In [5]:
topics_2017 = utils.get_topics('../input_files/2017.topics.xml')
topics_2017.head(10)

,topic,disease,gene,gene1,gene2,gene3,sex,age,age_group
1,1,Liposarcoma,CDK4 Amplification,CDK4 Amplification,,,male,38,adult
2,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",KRAS (G13D),BRAF (V600E),,male,52,aged
3,3,Meningioma,"NF2 (K322), AKT1(E17K)",NF2 (K322),AKT1(E17K),,female,45,aged
4,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",FGFR1 Amplification,PTEN (Q171),,female,67,aged
5,5,Melanoma,"BRAF (V600E), CDKN2A Deletion",BRAF (V600E),CDKN2A Deletion,,female,45,aged
6,6,Melanoma,NRAS (Q61K),NRAS (Q61K),,,male,55,aged
7,7,Lung cancer,EGFR (L858R),EGFR (L858R),,,female,50,aged
8,8,Lung cancer,EML4-ALK Fusion transcript,EML4-ALK Fusion transcript,,,male,52,aged
9,9,Gastrointestinal stromal tumor,KIT Exon 9 (A502_Y503dup),KIT Exon 9 (A502_Y503dup),,,female,49,aged
10,10,Lung adenocarcinoma,KRAS (G12C),KRAS (G12C),,,female,61,aged


The aim of the competition is to produce a file with relevant results for all topics (maximum: 1000 per topic). That means, to find the top 1000 most relevant documents (could be less) from PubMed for each topic. This is called a **run in trec_eval format**. Each group con submit up to 5 runs for each subtask (5 for abstracts, 5 for clinical trials). Below is an example we submitted in 2017. **Note than in 2019 there will be 3 new columns after the run name to add treatments**.

In [4]:
example_run = utils.from_trec_run_file('../../archived/2017/submitted/mugpubbase.trec_results')
example_run.sort_values(by=['TOPIC_NO', 'RANK']).head(10)

,TOPIC_NO,Q0,ID,RANK,SCORE,RUN_NAME
16249,1,Q0,25028469,1,65.013748,mugpubbase
16250,1,Q0,23852861,2,64.275940,mugpubbase
16251,1,Q0,26643872,3,60.410633,mugpubbase
16252,1,Q0,26336885,4,59.411972,mugpubbase
16253,1,Q0,25121597,5,58.291725,mugpubbase
16254,1,Q0,11505267,6,57.617733,mugpubbase
16255,1,Q0,24065146,7,55.737698,mugpubbase
16256,1,Q0,25059573,8,55.404232,mugpubbase
16257,1,Q0,26642065,9,54.019871,mugpubbase
16258,1,Q0,23569312,10,52.573662,mugpubbase


This means that according to our experiments the most relevant document on pubmed to treat **topic 1** (a 38 male with Liposarcoma and CDK4 amplification) is the article with [PubMed ID 25028469](https://www.ncbi.nlm.nih.gov/pubmed/?term=25028469), and so on.

The file above will be the evaluated against a gold standard (generated with results submitted by participants, so never available for training that year). This file is called a **qrels** file . There are 3 relevance scores: **2=relevant**, **1=partially relevant**, **0=not relevant**.

In [5]:
qrels_2017 = utils.get_qrels('../input_files/2017.abstracts.sorted.qrels')
qrels_2017.head(10)

,topic,doc_id,relev
0,1,10755400,2
1,1,10981874,2
2,1,11369052,2
3,1,11505267,2
4,1,12242528,2
5,1,14736600,2
6,1,15154619,2
7,1,15991843,2
8,1,17214366,2
9,1,17895758,2


The ideal situation would be that our submitted file matches exactly the relevance order of the qrels file. That is that all most relevant documents (relev=2) are ranked first, etc. The program [trec_eval](https://github.com/usnistgov/trec_eval) is used by the organization to evaluate the results using a bunch of metrics. This program is written in C and needs to be downloaded and compiled.

Fortunately, there is python package called **pytrec_eval** that provides the same functionality, which I also improved to make it easier to use and focus only on the evaluation measures for the competition.

In [6]:
results, aggregated = evaluation.evaluate(qrels_2017, example_run)
aggregated

{'recall_1000': 0.5224, 'ndcg': 0.4956, 'Rprec': 0.2772, 'P_10': 0.62}

Remember, the final aim is to generate (up to 5) run files like *example_run* using different strategies and submit them to the competition. The usual way is to use the strategies on topics and qrels from 2017 (the only year we have a gold standard available), then re-run them in the topics of the current year and submit them. However, note than in 2019 the competition is slightly different and now [treatments are also requested (optionally)](http://www.trec-cds.org/2019.html).

How to generate the qrels files given our elasticsearch server? Some hints are below and in the abstracts_experiments notebook accompanying this one. The main idea is that each strategy maps to an elasticsearch query and the retrieved results are transformed into a trec_eval file and submitted (also very easily using utilities I wrote). The queries are a collection of strategies from previous years and experimenting.

# FIXME: OLD STUFF FROM 2018

In [7]:
topics_2018=utils.get_topics('../input_files/2018.topics.xml')

## SUBMITTED RUN 1
Baseline query with MeSH age groups and grid-search optimized parameters.
Relevance: 3

Evaluate on 2017 topics and qrels first.

In [8]:
run_params = {
    'run_id':'imi_mug_abs1',
    'query_template':'submitted/imi_mug_abs1.json',
}

run, run_params = running.run(topics_2017, 'ABSTRACTS', run_params)
results, aggregated = evaluation.evaluate(qrels_2017, run)

aggregated

RUN: imi_mug_abs1 TOPICS: 30


{'recall_1000': 0.5366, 'ndcg': 0.4731, 'Rprec': 0.2392, 'P_10': 0.4467}

Run on 2018 topics and export to file for submission

In [ ]:
run, run_params = running.run(topics_2018, 'ABSTRACTS', run_params)
run

RUN: imi_mug_abs1 TOPICS: 50


In [ ]:
utils.to_trec_run_file(run, run_params)

## SUBMITTED RUN 2

Baseline query with MeSH age groups and grid-search optimized parameters. Boost English and extra topics.
Relevance: 2

Evaluate on 2017 topics and qrels first.

In [ ]:
run_params = {
    'run_id':'imi_mug_abs2',
    'query_template':'submitted/imi_mug_abs2.json',
}

run, run_params = running.run(topics_2017, 'ABSTRACTS', run_params)
results, aggregated = evaluation.evaluate(qrels_2017, run)

aggregated

Run on 2018 topics and export to file for submission

In [ ]:
run, run_params = running.run(topics_2018, 'ABSTRACTS', run_params)
run

In [ ]:
utils.to_trec_run_file(run, run_params)

## SUBMITTED RUN 3

Baseline query with MeSH age groups and grid-search optimized parameters. Boost English and extra topics. Search age group and sex in title, abstract, and MeSH.
Relevance: 4

Evaluate on 2017 topics and qrels first.

In [ ]:
run_params = {
    'run_id':'imi_mug_abs3',
    'query_template':'submitted/imi_mug_abs3.json',
}

run, run_params = running.run(topics_2017, 'ABSTRACTS', run_params)
results, aggregated = evaluation.evaluate(qrels_2017, run)

aggregated

Run on 2018 topics and export to file for submission

In [ ]:
run, run_params = running.run(topics_2018, 'ABSTRACTS', run_params)
run

In [ ]:
utils.to_trec_run_file(run, run_params)

## SUBMITTED RUN 4

Baseline query with MeSH age groups and grid-search optimized parameters. Boost English and extra topics. Search age group and sex in title, abstract, and MeSH. Abstract should exist.
Relevance: 5

Evaluate on 2017 topics and qrels first.

In [ ]:
run_params = {
    'run_id':'imi_mug_abs4',
    'query_template':'submitted/imi_mug_abs4.json',
}

run, run_params = running.run(topics_2017, 'ABSTRACTS', run_params)
results, aggregated = evaluation.evaluate(qrels_2017, run)

aggregated

Run on 2018 topics and export to file for submission

In [ ]:
run, run_params = running.run(topics_2018, 'ABSTRACTS', run_params)
run

In [ ]:
utils.to_trec_run_file(run, run_params)

## SUBMITTED RUN 5

Baseline query with MeSH age groups and grid-search optimized parameters. Boost English and extra topics. Abstract should exist.
Relevance: 1

Evaluate on 2017 topics and qrels first.

In [ ]:
run_params = {
    'run_id':'imi_mug_abs5',
    'query_template':'submitted/imi_mug_abs5.json',
}

run, run_params = running.run(topics_2017, 'ABSTRACTS', run_params)
results, aggregated = evaluation.evaluate(qrels_2017, run)

aggregated

Run on 2018 topics and export to file for submission

In [ ]:
run, run_params = running.run(topics_2018, 'ABSTRACTS', run_params)
run

In [ ]:
utils.to_trec_run_file(run, run_params)